In [9]:
from __future__ import division
import numpy as np
import pandas as pd
import os
import glob
import tensorflow as tf
#from skimage import io
#from skimage.util import img_as_float, img_as_ubyte
#import matplotlib.cm as cm
#from matplotlib import pyplot as plt
#%matplotlib inline
import time
from six.moves import xrange 

In [10]:
data = pd.read_pickle('../Data/mean_normalised_df_no_class_bias.pkl') 
mean_img = pd.read_pickle('../Data/mean_img_no_class_bias.pkl')

In [17]:
PATCH_DIM = 31
BATCH_SIZE = 60
NUM_CLASSES = 2

In [18]:
def next_batch(size, df, current_batch_ind):
    """Returns the next mini batch of data from the dataset passed
    
    Args:
        size: length of the current requested mini batch
        df: the data set consisting of the images and the labels
        current_batch_ind: the current position of the index in the dataset
    
    Returns:
        (batch_x, batch_y): A tuple of np arrays of dimensions 
                        [size, patch_dim**2*3] and [size, NUM_CLASSES] respectively 
        current_batch_ind: the updated current position of the index in the dataset
        df: when the requested batch_size+current_batch_ind is more than the length of the data set,
            the data is shuffled again and current_batch_ind is reset to 0, and this new data set is
            returned
    
    """
    
    if current_batch_ind + size> len(df):
        current_batch_ind = 0
        df = df.iloc[np.random.permutation(len(df))]
        df = df.reset_index(drop=True)
        
    batch_x = np.zeros((size, PATCH_DIM**2*3))
    batch_y = np.zeros((size, NUM_CLASSES), dtype = 'uint8')
    for i in range(current_batch_ind, current_batch_ind+size):
        batch_x[i - current_batch_ind] = df.loc[i][:-1]
        batch_y[i - current_batch_ind][int(df.loc[i][PATCH_DIM**2*3])]=1
        
    current_batch_ind += size  
    return (batch_x, batch_y), current_batch_ind, df

In [19]:
def inference(images, keep_prob, fc_hidden_units1=512):
    """ Builds the model as far as is required for running the network
    forward to make predictions.

    Args:
        images: Images placeholder, from inputs().
        keep_prob: Probability used for Droupout in the final Affine Layer
        fc_hidden_units1: Number of hidden neurons in final Affine layer
    Returns:
        softmax_linear: Output tensor with the computed logits.
    """
    with tf.variable_scope('h_conv1') as scope:
        weights = tf.get_variable('weights', shape=[4, 4, 3, 64], 
                                  initializer=tf.contrib.layers.xavier_initializer_conv2d())
        biases = tf.get_variable('biases', shape=[64], initializer=tf.constant_initializer(0.05))
        
        # Flattening the 3D image into a 1D array
        x_image = tf.reshape(images, [-1,PATCH_DIM,PATCH_DIM,3])
        z = tf.nn.conv2d(x_image, weights, strides=[1, 1, 1, 1], padding='VALID')
        h_conv1 = tf.nn.relu(z+biases, name=scope.name)
    with tf.variable_scope('h_conv2') as scope:
        weights = tf.get_variable('weights', shape=[4, 4, 64, 64], 
                                  initializer=tf.contrib.layers.xavier_initializer_conv2d())
        biases = tf.get_variable('biases', shape=[64], initializer=tf.constant_initializer(0.05))
        z = tf.nn.conv2d(h_conv1, weights, strides=[1, 1, 1, 1], padding='SAME')
        h_conv2 = tf.nn.relu(z+biases, name=scope.name)
    
    h_pool1 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME', name='h_pool1')
    
    with tf.variable_scope('h_conv3') as scope:
        weights = tf.get_variable('weights', shape=[4, 4, 64, 64], 
                                  initializer=tf.contrib.layers.xavier_initializer_conv2d())
        biases = tf.get_variable('biases', shape=[64], initializer=tf.constant_initializer(0.05))
        z = tf.nn.conv2d(h_pool1, weights, strides=[1, 1, 1, 1], padding='SAME')
        h_conv3 = tf.nn.relu(z+biases, name=scope.name)
        
    h_pool2 = tf.nn.max_pool(h_conv3, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME', name='h_pool2')
    
    with tf.variable_scope('h_fc1') as scope:
        weights = tf.get_variable('weights', shape=[7**2*64, fc_hidden_units1], 
                                  initializer=tf.contrib.layers.xavier_initializer())
        biases = tf.get_variable('biases', shape=[fc_hidden_units1], initializer=tf.constant_initializer(0.05))
        h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
        
        h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, weights) + biases, name = 'h_fc1')
        h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
        
        
    with tf.variable_scope('h_fc2') as scope:
        weights = tf.get_variable('weights', shape=[fc_hidden_units1, NUM_CLASSES], 
                                  initializer=tf.contrib.layers.xavier_initializer())
        biases = tf.get_variable('biases', shape=[NUM_CLASSES])
        
        logits = (tf.matmul(h_fc1_drop, weights) + biases)
    return logits

In [23]:
def calc_loss(logits, labels):
    """Calculates the loss from the logits and the labels.
    Args:
        logits: Logits tensor, float - [batch_size, NUM_CLASSES].
        labels: Labels tensor, int32 - [batch_size].
    Returns:
        loss: Loss tensor of type float.
    """
    labels = tf.to_float(labels)
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits, labels, name='xentropy')
    loss = tf.reduce_mean(cross_entropy, name='xentropy_mean')
    return loss

In [20]:
def placeholder_inputs(batch_size):
    """Generate placeholder variables to represent the input tensors.
    
    These placeholders are used as inputs by the rest of the model building
    code and will be fed from the downloaded data in the .run() loop, below.
    Args:
        batch_size: The batch size will be baked into both placeholders.
    Returns:
        images_placeholder: Images placeholder.
        labels_placeholder: Labels placeholder.
    """
    # Note that the shapes of the placeholders match the shapes of the full
    # image and label tensors, except the first dimension is now batch_size
    # rather than the full size of the train or test data sets.
    images_placeholder = tf.placeholder(tf.float32, shape=(batch_size, PATCH_DIM**2*3))
    labels_placeholder = tf.placeholder(tf.int32, shape=(batch_size, NUM_CLASSES))
    return images_placeholder, labels_placeholder

In [21]:
#UPDATE current_img_ind
def fill_feed_dict(data_set, images_pl, labels_pl, current_img_ind, batch_size):
    """Fills the feed_dict for training the given step.
    A feed_dict takes the form of:
    feed_dict = {
                  <placeholder>: <tensor of values to be passed for placeholder>,
                  ....
                }
    Args:
        data_set: The set of images and labels, from input_data.read_data_sets()
        images_pl: The images placeholder, from placeholder_inputs().
        labels_pl: The labels placeholder, from placeholder_inputs().
        current_img_ind: The current position of the index in the dataset
    Returns:
        feed_dict: The feed dictionary mapping from placeholders to values.
        current_img_ind: The updated position of the index in the dataset
        data_set: updated data_set
    """
    # Create the feed_dict for the placeholders filled with the next
    # `batch size ` examples.
    batch, current_img_ind, data_set= next_batch(batch_size, data_set, current_img_ind)

    feed_dict = {
      images_pl: batch[0],
      labels_pl: batch[1],
    }
    return feed_dict, current_img_ind, data_set


In [28]:
with tf.Graph().as_default():
    # Generate placeholders for the images and labels.
    images_placeholder, labels_placeholder = placeholder_inputs(BATCH_SIZE)

    # Build a Graph that computes predictions from the inference model.
    logits = inference(images_placeholder, 0.5, 512)
    loss = calc_loss(logits, labels_placeholder)

    # Create a saver for writing training checkpoints.
    saver = tf.train.Saver()

    # Create a session for running Ops on the Graph.
    with tf.Session() as sess:
        current_img_ind = 0
        start_time = time.time()
        saver.restore(sess, '../Data/model.ckpt')
        
        # Fill a feed dictionary with the actual set of images and labels
        # for this particular training step.
        feed_dict, current_img_ind, data = fill_feed_dict(data,
                                 images_placeholder,
                                 labels_placeholder, current_img_ind=current_img_ind, 
                                                        batch_size=BATCH_SIZE)

        predictions, loss_value = sess.run([logits, loss],
                               feed_dict=feed_dict)

        duration = time.time() - start_time
        print predictions.shape
        
        print loss_value
        

(60, 2)
0.0704492
